In [1]:
sandbox = 'fesc'
PATH_DATA = '/data/sandboxes/'+sandbox+'/data/'



# Repaso de Spark

Cargamos un dataset con información de la supervivencia de los pasajeros abordo del Titanic. El objetivo es, una vez limpiado el dataset, estudiarlo para extraer el máximo número de insights de los datos.



## Información sobre las variables del dataset

* **Name**: Nombre pasajero. Los títulos que tenían los pasajeros eran: 
    * 'Mr': Sr.
    * 'Miss': Señorita
    * 'Mrs': Sra.
    * 'Master': Señorito
    * 'Mlle': Señorita (Mademoiselle)
    * 'Ms': Señorita
    * 'Mme': Señora (Madame)
    * 'Lady': Señora
    * 'Countess': Condesa
    * 'Capt': Capitán
    * 'Col': Coronel
    * 'Don': Don
    * 'Dr': Dr.
    * 'Major': Comandante
    * 'Rev': Reverendo
    * 'Sir': Señor
    * 'Jonkheer': Joven Señor (Países Bajos)
    * 'Dona': Doña
    
* **PassengerID**: ID pasajero
* **Survived**: ¿Sobrevivió? (0 = No, 1 = Sí)
* **Pclass**: Clase billete (1 = 1ª, 2 = 2ª, 3 = 3ª)
* **Age**: Edad en años
* **Sibsp**: # of hermanos / maridos / mujeres abordo del Titanic (no se contabilizan amantes ni prometidas)
* **Parch**: # of padres / hijos abordo del Titanic (algunos niños viajaban solo con una niñera, para ellos Parch=0)
* **Ticket**: Número billete
* **Fare**: Tarifa pasajero
* **Cabin**: Número de camarote (La letra se refiere a la cubierta; los números son el camarote)
* **Embarked**: Puerto de embarcación (C = Cherbourg, Q = Queenstown, S = Southampton)






## Lectura de datos

In [2]:
from pyspark.sql import functions as F
import numpy as np
import pandas as pd

In [3]:
df = spark.read.csv(PATH_DATA + '/titanic.csv', sep=',', header=True, inferSchema=True)



### Comprobar lectura correcta del dataset

In [4]:
# Respuesta
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [5]:
# Respuesta
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower().replace(' ', '_').replace('.', ''))

In [6]:
# Respuesta
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|passengerid|survived|pclass|                name|   sex| age|sibsp|parch|          ticket|   fare|cabin|embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      



### Comprobar dimensiones del dataset



### Check dataset dimensions

In [7]:
# Respuesta
cols=len(df.columns)
rows=df.count()

print('Número de columnas: ',cols)
print('Número de filas: ', rows)

Número de columnas:  12
Número de filas:  891




### Estudiar balanceo del dataset (sobre la variable target)

In [25]:
# Respuesta
balance = df.groupBy('survived').count()
balance.show()

+--------+-----+
|survived|count|
+--------+-----+
|       0|  549|
|       1|  342|
+--------+-----+



In [23]:
# Respuesta
rows = df.count()

In [24]:
# Respuesta
balance.withColumn('proporcion', F.col('count') / rows * 100).show()

+--------+-----+-----------------+
|survived|count|       proporcion|
+--------+-----+-----------------+
|       0|  549|61.61616161616161|
|       1|  342|38.38383838383838|
+--------+-----+-----------------+





## Data Wrangling



### Estudio de los formatos de las variables

In [9]:
# Respuesta
df.printSchema()

root
 |-- passengerid: integer (nullable = true)
 |-- survived: integer (nullable = true)
 |-- pclass: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)



In [26]:
# Respuesta
df = df.withColumn('passengerid', F.col('passengerid').cast('string'))
df = df.withColumn('survived', F.col('survived').cast('string'))
df = df.withColumn('pclass', F.col('pclass').cast('string'))
df = df.withColumn('age', F.col('age').cast('integer'))

In [27]:
# Respuesta
categorial_columns = [c for c,t in df.dtypes if t in ['string', 'bool']]
categorial_columns

['passengerid',
 'survived',
 'pclass',
 'name',
 'sex',
 'ticket',
 'cabin',
 'embarked']

In [28]:
# Respuesta
numerical_columns = [c for c,t in df.dtypes if t in ['int', 'double']]
numerical_columns

['age', 'sibsp', 'parch', 'fare']

In [29]:
# Respuesta
df.printSchema()

root
 |-- passengerid: string (nullable = true)
 |-- survived: string (nullable = true)
 |-- pclass: string (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)





### Comprobación y eliminación de registros repetidos

In [14]:
# Respuesta
def show_duplicates(df, subset):  
    agg_count = df.groupBy(subset).count().filter(F.col('count') > 1)
    return agg_count

In [15]:
# Respuesta
duplicates = show_duplicates(df, df.columns)
duplicates.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
|passengerid|survived|pclass|name|sex|age|sibsp|parch|ticket|fare|cabin|embarked|count|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-----+



In [16]:
# Respuesta
duplicates = show_duplicates(df, ['sex'])
duplicates.show()

+------+-----+
|   sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+

